In [ ]:
import pandas as pd
import numpy as np
import sys
from os import getcwd
from os.path import join

data_path = "/home/davina/Private/crrt-data"
ref = "df_[startdate-7d,startdate].parquet"
f = "df_[startdate+1-7d,startdate+1].parquet"

In [ ]:
ref_df = pd.read_parquet(join(data_path, ref))
f_df = pd.read_parquet(join(data_path, f))

In [ ]:
ref_df.equals(f_df)

In [ ]:
ref_df.shape

In [ ]:
for i in range(1, 8):
    file = f"df_[startdate+{i}-7d,startdate+{i}].parquet"
    f_df = pd.read_parquet(join(data_path, file))
    print(f_df.shape)

# Column Asymmetry across slided window

In [ ]:
# in ref but not other
ref_df.columns.difference(f_df.columns)

In [ ]:
f_df.columns.difference(ref_df.columns)

In [ ]:
ref_df.columns

# Explore if sliding windows actually gives me the correct range of dates

In [ ]:
sys.path.insert(0, join(getcwd(), "../../module_code"))

from data.load import load_outcomes
from data.longitudinal_utils import get_time_window_mask, apply_time_window_mask

In [ ]:
dx = pd.read_csv(join(data_path, "Encounter_Diagnoses.txt"))

In [ ]:
outcomes_df = load_outcomes(data_path, group_by=["IP_PATIENT_ID", "Start Date"])
time_window = get_time_window_mask(
        outcomes_df, {"DAYS": 7}, None, "Start Date", slide_window_by=2
    )

In [ ]:
masked_dx = apply_time_window_mask(dx, "DIAGNOSIS_DATE", time_window)

In [ ]:
diff = masked_dx["Start Date"] - pd.to_datetime(masked_dx["DIAGNOSIS_DATE"])
print(diff[diff < np.timedelta64(0, "D")].shape)
print(diff[diff < np.timedelta64(-1, "D")].shape)
print(diff[diff > np.timedelta64(7-2, "D")].shape)
print(diff[diff > np.timedelta64(7-3, "D")].shape)

In [ ]:
outcomes_df["End Date"] - np.timedelta64(1, "D")

In [ ]:
print(f"{(outcomes_df['CRRT Total Days'] <= 7).sum()} patients with leq 7 days data.")
print(f"{(outcomes_df['CRRT Total Days'] > 7).sum()} patients with more than 7 days data.")

In [ ]:
outcomes_over_7d = outcomes_df[outcomes_df["CRRT Total Days"] > 7]["recommend_crrt"]
f"{outcomes_over_7d.sum()} pts recommend crrt out of {len(outcomes_over_7d)}, ({outcomes_over_7d.mean()*100}%)."